In [13]:
import os


In [43]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [16]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class TraningConfig:
    root_dir: Path
    training_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool#
    params_image_size: list
    


In [18]:
os.chdir("CHICKEN-DISEASE-CLASSIFICATION")

In [19]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [48]:
from clasifier_cnn.constants import *
from clasifier_cnn.utils.common_func import read_yaml, create_directories

import tensorflow as tf 




In [26]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir  = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)]
        )

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TraningConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TraningConfig(
            root_dir =  Path(training.root_dir),
            training_model_path = Path(training.training_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data =  Path(training_data),
            params_epoch =  params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation =  params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config


        


In [49]:
from tensorflow.keras.optimizers import Adam

In [27]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time



In [28]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_checkpoint_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]


In [ ]:
from tensorflow.keras.optimizers import Adam

self.model.compile(
    optimizer=Adam(learning_rate=0.001),  # <- new instance
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [58]:
class Training:
    def __init__(self, config:TraningConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "biliniear"
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **data_generator_kwargs
            )
        else:
            train_datagenerator = valid_generator

        self.train_datagenerator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_datagenerator.samples // self.train_datagenerator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.compile(
        optimizer=Adam(learning_rate=0.001),  # <- new instance
        loss='categorical_crossentropy',
        metrics=['accuracy']
        
        )
        
        self.model.fit(
            self.train_datagenerator,
            epochs = self.config.params_epoch,
            steps_per_epoch =  self.steps_per_epoch,
            validation_steps =  self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.training_model_path,
            model = self.model
        )

In [59]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_checkpoint_callbacks()

    training_config = config.get_training_config()
    training = Training(config =training_config )
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-05-09 17:39:35,175: INFO: common_func: Yaml file: config/config.yaml has been loaded succesfully]
[2025-05-09 17:39:35,180: INFO: common_func: Yaml file: params.yaml has been loaded succesfully]
[2025-05-09 17:39:35,182: INFO: common_func: created directory at: artifacts]
[2025-05-09 17:39:35,183: INFO: common_func: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-05-09 17:39:35,184: INFO: common_func: created directory at: artifacts/prepare_callbacks/tensorflow_log_dir]
[2025-05-09 17:39:35,186: INFO: common_func: created directory at: artifacts/training]
[2025-05-09 17:39:35,374: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.6372 - loss: 0.8469[2025-05-09 17:42:35,136: WARNING: saving_api: You are 